In [22]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch

In [12]:
protein_sequence = "MGQPGNGSAFLLAPNGSHAPDHDVTQERDEVWVVGMGIVMSLIVLAIVFGNVLVITAIAKFERLQTVTNYFITSLACADLVMGLAVVPFGAAHILMKMWTFGNFWCEFWTSIDVLCVTASIETLCVIAVDRYFAITSPFKYQSLLTKNKARVIILMVWIVSGLTSFLPIQMHWYRATHQEAINCYANETCCDFFTNQAYAIASSIVSFYVPLVIMVFVYSRVFQEAKRQLQKIDKSEGRFHVQNLSQVEQDGRTGHGLRRSSKFCLKEHKALKTLGIIMGTFTLCWLPFFIVNIVHVIQDNLIRKEVYILLNWIGYVNSGFNPLIYCRSPDFRIAFQELLCLRRSSLKAYGNGYSSNGNTGEQSGYHVEQEKENKLLCEDLPGTEDFVGHQGTVPSDNIDSQGRNCSTNDSLL"

In [13]:
protein = ESMProtein(sequence=protein_sequence)
client = ESMC.from_pretrained("esmc_300m").to("cpu")
protein_tensor = client.encode(protein)
logits_output = client.logits(
    protein_tensor, LogitsConfig(sequence=True, return_embeddings=True)
)
print(logits_output.logits, logits_output.embeddings)

ForwardTrackData(sequence=tensor([[[-37.7603, -37.7463, -37.7816,  ..., -37.7437, -37.7768, -37.7802],
         [-42.1356, -42.0509, -42.1101,  ..., -42.0273, -42.0702, -42.1130],
         [-39.7645, -39.7032, -39.7317,  ..., -39.7216, -39.6800, -39.7218],
         ...,
         [-39.9428, -39.8964, -39.9847,  ..., -39.9400, -39.9402, -39.9605],
         [-36.2157, -36.2523, -36.2886,  ..., -36.2580, -36.2593, -36.2999],
         [-35.5685, -35.5536, -35.6612,  ..., -35.5273, -35.6028, -35.6262]]]), structure=None, secondary_structure=None, sasa=None, function=None) tensor([[[ 0.0043,  0.0036,  0.0008,  ...,  0.0041, -0.0008, -0.0167],
         [ 0.0072,  0.0534,  0.0499,  ..., -0.0087, -0.0335, -0.0042],
         [-0.0184,  0.0335,  0.0410,  ..., -0.0532, -0.0594,  0.0030],
         ...,
         [-0.0086,  0.0272,  0.0095,  ...,  0.0134,  0.0014,  0.0345],
         [-0.0270, -0.0347, -0.0075,  ...,  0.0333, -0.0380,  0.0006],
         [ 0.0048,  0.0244,  0.0096,  ..., -0.0064, -0.055

In [25]:
# take average of all amino acid sequences
tensor = logits_output.embeddings[0]
col_means = torch.mean(tensor, dim=0)
tensor = col_means.unsqueeze(0)
tensor.shape


# save tensor
torch.save(tensor, 'esmc_embedding')


torch.Size([1, 960])

In [ ]:
#TODO: make this a class pipeline, process all single AA mutations through it